
Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# Mem Set Range Gradients (Activations and Params)

In [ ]:
#@title Import libraries
import transformer_lens, torch, gc, itertools, functools, math, glob, tqdm
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict 

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LogNorm, SymLogNorm

mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.left'] = False
mpl.rcParams['axes.spines.bottom'] = False

import sys
sys.path.append('/home/jupyter/')
from paraMem.utils import modelHandlers, dataLoaders, gradient, localizing, patching

torch.set_printoptions(precision=2)

## Model

In [ ]:
model_type = "gpt-neo-125M"
model = modelHandlers.load_model(model_type=model_type, DEVICE="cpu")
modelHandlers.set_no_grad(model, ["embed", "pos_embed", "unembed"])

## Load Data

In [ ]:
range_data = dataLoaders.load_pile_splits(f"{model_type}/preds", file_names=None)
range_dl = list(map(lambda toks_NI: torch.utils.data.DataLoader(toks_NI, batch_size=50, shuffle=False), range_data))

### Run backprop and gather gradients

In [ ]:
def collect_pool_activs(model, cache:dict=None, c_type:str=None, I_range=None):
    vals, names = localizing.collect_c_type(model=model, cache=cache, c_type=c_type)
    pool_vals = localizing.pool_tensor(vals, pool="max", abs_vals=True, topP=0.01, norm_by_entries=False)
    
    ## consider either all tokens or only selected token, then mean over sequences    
    if I_range is not None: 
        pool_vals = pool_vals[:,I_range[0]:I_range[1]]
    pool_vals = pool_vals.mean(1) ## pool over tokens
    pool_vals = pool_vals.mean(0) ## pool over seqs
            
    ## reshape for plotting
    names = list()
    if len(pool_vals.shape) == 2: ## attention
        names = [f"{c_type} H{i}" for i in range(0,pool_vals.shape[1])]
    else: ## mlp
        pool_vals = pool_vals.unsqueeze(-1)
        names = [f"{c_type}"]
    return pool_vals, names


def gather_activation_grads(model, cache:dict, I_range:list=[50,100], c_types:list=["q", "k", "v", "z", "pre", "post"]):  #"q", "k", "v", "z", "pre", "post", "attn_out", "mlp_out"
    vals, names = [], []
    for c_type in c_types:
        c_type_vals, c_type_names = collect_pool_activs(model=model, cache=cache, c_type=c_type, I_range=I_range)
        vals.append(c_type_vals)
        names += c_type_names
    vals = torch.cat(vals, dim=-1)
    return vals, names


def gather_param_grads(model, I_range:list=[50,100], c_types:list=["W_K","W_Q","W_V","W_O","W_in","W_out"]):
    param_vals, param_names = [], []
    for c_type in c_types:
        vals, names = localizing.collect_c_type(model=model, cache=None, c_type=c_type) 
        if c_type in ['W_Q', 'W_K', 'W_V', 'W_O']: ## attention
            vals = vals.view(vals.shape[0], vals.shape[1], -1)
            names = [f"{names[0].split('.')[-1]} H{i}" for i in range(vals.shape[1])]
        else: ## mlps
            vals = vals.view(vals.shape[0], 1, -1)
            names = [f"{names[0].split('.')[-1]}"]
        pool_vals = localizing.pool_tensor(vals, pool="max", abs_vals=True, topP=0.1, norm_by_entries=False)
        param_vals.append(pool_vals)
        param_names += names
    param_vals = torch.cat(param_vals, dim=1)
    return param_vals, param_names


def gather_param_stats(model, param_stats, i, I_range:list=[49,99], c_types:list=["W_K","W_Q","W_V","W_O","W_in","W_out"]):
    
    param_stats["attn"][i] = {l: 0.0 for l in range(model.cfg.n_layers)}
    param_stats["mlp"][i] = {l: 0.0 for l in range(model.cfg.n_layers)}

    for c_type in c_types:
        vals, names = localizing.collect_c_type(model=model, cache=None, c_type=c_type) 
        layer_sum = torch.reshape(torch.abs(vals), (model.cfg.n_layers,-1)).sum(-1)
        param_stats["sum"][i][c_type] = torch.abs(vals).sum().item()
        #param_stats["mean"][c_type] = vals.mean().item()
        param_stats["var"][i][c_type] = vals.var().item() 
        param_stats["max"][i][c_type] = vals.max().item()
        param_stats["min"][i][c_type] = torch.abs(vals.min()).item()
        
        if c_type in ["W_K","W_V","W_Q","W_O"]:
            for l in range(model.cfg.n_layers):
                param_stats["attn"][i][l] += layer_sum[l].item()
        elif c_type in ["W_in","W_out"]:
            for l in range(model.cfg.n_layers):
                param_stats["mlp"][i][l] += layer_sum[l].item()        
        #stats = {"abs_sum": torch.abs(vals).sum().item(), "mean": vals.mean().item(), "var": vals.var().item(), "max": vals.max().item(), "min": torch.abs(vals.min()).item(), "layer_abs_sum":layer_sum}
    return param_stats

def gradient_stats(range_pool_vals:list):
    total_grad = []
    total_grad_per_layer = []
    total_grad_var = []
    for i, vals in enumerate(range_pool_vals):
        total_grad.append(vals.sum())
        grad_per_layer = [f"{x:.2f}" for x in vals.sum(-1)]
        total_grad_per_layer.append(grad_per_layer)
        total_grad_var.append(torch.var(vals).item())
    return total_grad, total_grad_per_layer, total_grad_var


In [ ]:
fwd_bwd = "bwd"
grad_type = "params"

#I_range=[0,99]
I_range=[49,99]

range_pool_vals = []
param_stats = {"sum": defaultdict(dict), "var": defaultdict(dict), "max": defaultdict(dict), "min": defaultdict(dict), "attn":defaultdict(dict), "mlp":defaultdict(dict)}

for i, dl in tqdm.tqdm(enumerate(range_dl[:])):
    toks_NI = next(iter(dl))
    metric = functools.partial(gradient.single_seq_metric, NI_idcs=I_range)
    fwd_cache, bwd_cache, _ = gradient.run_single_fwd_bwd(model, metric_fn=metric, c_toks_NI=toks_NI)
    
    ## POOL
    if grad_type=="activs":
        pool_vals, names = gather_activation_grads(model, {"fwd":fwd_cache,"bwd":bwd_cache}[fwd_bwd], I_range=I_range)
    elif grad_type=="params":
        pool_vals, names = gather_param_grads(model, I_range=I_range)
        param_stats = gather_param_stats(model, param_stats, i, I_range=I_range)
    range_pool_vals.append(pool_vals)
    
total_grad, total_grad_per_layer, total_grad_var = gradient_stats(range_pool_vals)

## Plot Heatmap

In [ ]:
fontsize = 12
fig, axs = plt.subplots(len(range_pool_vals), 1, figsize=(16, 14), gridspec_kw={'hspace': 0.5})

#plot_types = ['EM 0-10', 'EM 11-49', 'EM 50']
plot_types = ['EM 0-10', 'EM 11-29', 'EM 30-49', 'EM 50']


for i, ax in enumerate(axs):
    if i == 0:
        cmap = "Blues"
    elif i == 3:
        cmap = "Reds"
    else:
        cmap = "binary"
    
    plot_vals = range_pool_vals[i].numpy()
    #sns.heatmap(plot_vals[1:-1,:],cmap=mpl.colormaps[cmap],center=None,xticklabels=names,yticklabels=np.arange(1,plot_vals.shape[0]-1),square=False,ax=ax)# norm=SymLogNorm(linthresh=1.0))
    sns.heatmap(plot_vals[:,:],cmap=mpl.colormaps[cmap],center=None,xticklabels=names,yticklabels=np.arange(0,plot_vals.shape[0]),square=False,ax=ax)#, norm=LogNorm())
    ax.invert_yaxis()
    ax.set_title(f"{plot_types[i]}: {fwd_bwd} {grad_type} grads for tokens {I_range[0]+1} to {I_range[1]+1} (grad sum: {total_grad[i]:.2f}, grad var: {total_grad_var[i]:.2e})", fontsize=fontsize, loc="left")
    ax.set_ylabel("layer", fontsize=fontsize)
    ax.tick_params(axis='y', which='major', labelsize=fontsize)
    ax.tick_params(axis='x', which='major', labelsize=fontsize-3)

    # Create a second y-axis on the right side
    ax2 = ax.twinx()
    ax2.set_yticks(np.arange(0,len(total_grad_per_layer[i])))
    ax2.set_yticklabels(total_grad_per_layer[i], fontsize = fontsize-4)
    ax2.set_ylabel("layer-wise gradient sum", fontsize = fontsize-4)
    
fig.savefig(f"{dataLoaders.ROOT}/results/{model_type}_{grad_type}_max_{I_range[0]}_{I_range[1]}.pdf", dpi=200, bbox_inches="tight")

## Plot Stats

In [ ]:
fontsize = 15
fig, axs = plt.subplots(len(param_stats.keys()), 1, figsize=(16, 5.5), gridspec_kw={'hspace': 0.5})

#plot_types = ['EM 0-10', 'EM 11-49', 'EM 50']
plot_types = ['EM 0-10', 'EM 11-29', 'EM 30-49', 'EM 50']

#cmap = plt.get_cmap('coolwarm')
#cmap_discrete = ListedColormap(cmap(np.linspace(0, 1, len(x_labels)))).colors
color = ["blue", "grey", "grey", "red"]
bar_width, group_gap = 0.9, 0.5

for i, (stat_name, range_param_dict) in enumerate(param_stats.items()):
    
    last_x = 0
    row_x, row_labels = [], []
    axs[i].tick_params(axis='both', which='major', labelsize=fontsize-4)
    axs[i].set_ylabel(stat_name, rotation=90, fontsize=fontsize)
    axs[i].yaxis.set_label_coords(-0.05, 0.5)
    for j, (range_key, param_value_dict) in enumerate(range_param_dict.items()):
        param_labels = list(param_value_dict.keys())
        x = np.arange(0, len(param_labels)) + last_x   
        row_x += list(x)
        row_labels += param_labels
        
        y = list(param_value_dict.values())
        axs[i].bar(x, y, bar_width, color=color[j])   
        
        if i==0: ## have x titles only in first row
            axs[0].text(last_x, 4500, plot_types[j], fontsize=fontsize, color=color[j], verticalalignment='bottom')#, transform=axs[0].transAxes)
        #if (i==1) or (i==2) or (i==3):
        axs[i].set_yscale("log")
        
        last_x += (len(x) + group_gap)
    
    axs[i].set_xlim(-1, row_x[-1]+1)
    axs[i].set_xticks(row_x) 
    axs[i].set_xticklabels(row_labels, rotation=0, fontsize=fontsize-6)
            
fig.savefig(f"{dataLoaders.ROOT}/results/{model_type}_{grad_type}_em_stats.pdf", dpi=200, bbox_inches="tight")

In [ ]:
attn_mlp_stats = {k:v for k,v in param_stats.items() if k in ['attn', 'mlp']}

fontsize = 15
fig, axs = plt.subplots(len(attn_mlp_stats.keys()), 1, figsize=(16, 2), gridspec_kw={'hspace': 0.5})

#plot_types = ['EM 0-10', 'EM 11-49', 'EM 50']
plot_types = ['EM 0-10', 'EM 11-29', 'EM 30-49', 'EM 50']

#cmap = plt.get_cmap('coolwarm')
#cmap_discrete = ListedColormap(cmap(np.linspace(0, 1, len(x_labels)))).colors
color = ["blue", "grey", "grey", "red"]
bar_width, group_gap = 0.9, 0.5

for i, (stat_name, range_param_dict) in enumerate(attn_mlp_stats.items()):
    
    last_x = 0
    row_x, row_labels = [], []
    axs[i].tick_params(axis='both', which='major', labelsize=fontsize-4)
    axs[i].set_ylabel(stat_name, rotation=90, fontsize=fontsize)
    axs[i].yaxis.set_label_coords(-0.05, 0.5)
    for j, (range_key, param_value_dict) in enumerate(range_param_dict.items()):
        param_labels = list(param_value_dict.keys())
        x = np.arange(0, len(param_labels)) + last_x   
        row_x += list(x)
        row_labels += param_labels
        
        y = list(param_value_dict.values())
        axs[i].bar(x, y, bar_width, color=color[j])   
        
        if i==0: ## have x titles only in first row
            axs[0].text(last_x, 4500, plot_types[j], fontsize=fontsize, color=color[j], verticalalignment='bottom')#, transform=axs[0].transAxes)
        #if (i==1) or (i==2) or (i==3):
        axs[i].set_yscale("log")
        
        last_x += (len(x) + group_gap)
    
    axs[i].set_xlim(-1, row_x[-1]+1)
    axs[i].set_xticks(row_x) 
    axs[i].set_xticklabels(row_labels, rotation=0, fontsize=fontsize-6)
            
fig.savefig(f"{dataLoaders.ROOT}/results/{grad_type}_em_stats.pdf", dpi=200, bbox_inches="tight")